# Chapter 08 웹 크롤링

## 8.2 웹 데이터 자동 수집

### HTML 소스 가져오기

In [ ]:
!pip install requests

In [ ]:
import requests												# requests 모듈 불러오기
res = requests.get("http://www.naver.com", verify = False)	# GET 방식으로 네이버 홈페이지 가져오기
print(res.text)												# 한글로 내용 출력
#print(res.content)											# uni-code로 내용 출력

In [ ]:
parameters = {"code" : "005930"}
res = requests.get("https://finance.naver.com/item/sise.nhn", params=parameters, verify = False)
print(res.url)
print(res.text)

In [ ]:
res = requests.get("https://finance.naver.com/item/sise.nhn?code=005930", verify = False)
print(res.text)

In [ ]:
#res = requests.post("https://finance.naver.com/item/sise.nhn = 005930", verify = False)
res = requests.post("https://finance.naver.com/item/sise.nhn?code =005930", verify = False)
print(res.text)

In [ ]:
res = requests.post("https://naver.com/", verify = False)
print(res.text)

### HTML 소스를 데이터로 변환하기

In [ ]:
!pip install bs4			# BeautifulSoup 설치 

In [ ]:
from bs4 import BeautifulSoup			# BeautifulSoup 모듈 불러오기

# html_txt 변수에 HTML 소스와 텍스트 입력
html_txt = "<p class='weather' id='tw'>오늘의 날씨</p>  <h1> 한때 소나기가 내리겠습니다.</h1>"

soup = BeautifulSoup(html_txt,"html.parser")	 	# 파싱하기, 분석 가능한 데이터 형태로 변환
soup

In [ ]:
tag = soup.find("p")		# p 태그를 찾아 tag에 저장

print(tag)					# tag 출력
print(tag.name)				# tag명 출력
print( tag.attrs )			# tag 속성 출력
print(tag.attrs["class"])	# tag 속성 중 class만 출력
print(tag.attrs["id"])		# tag 속성 중 id만 출력
print(tag.text)				# tag 내 텍스트 출력

### html 소스를 분석용 데이터로 파싱(변환)하기


In [ ]:
from bs4 import BeautifulSoup			# BeautifulSoup 모듈 불러오기
# html 소스 생성하기
html_txt = """
<html>
    <head><title>BS page</title></head>
    <body>
        <h1 class="portal_cls">검색포털</h1>
        <p>
            <a href="http://www.daum.net">다음 바로가기</a><br>
            <ul>
                <li>장미 축제
                    <a href="http://www.daum.net/rose">장미 축제 바로가기</a><br>
                <li>불꽃 축제
                    <a href="http://www.daum.net/fireworks">불꽃 축제 바로가기</a><br>
            </ul>
            <a href="http://www.naver.com">네이버 바로가기</a>
        </p>
        <a href="http://www.google.com" class="alink_cls">구글</a>
        <p class="footage_cls" id="company">2021, ABC Company </p>
        <p class="footage_cls" id="addr">Korea</p>
    </body>
</html>
"""
soup = BeautifulSoup(html_txt, "html.parser")		# 파싱하기, 분석 가능한 데이터 형태로 변환
soup

In [ ]:
tag = soup.select_one("h1")			# <h1> </h1> 태그 추출
print(tag.text)					# tag 안에 있는 텍스트만 출력

#### select() 함수로 태그를 추출하면 list형 자료로 반환되기 때문에 반복문을 활용해 데이터를 하나씩 가져와야 함


In [ ]:
tag_list = soup.select("h1")		# <h1> ~ </h1> 태그 모두 찾기
for tag in tag_list:				# tag_list 내 데이터를 하나씩 읽어오기
    print(tag.text)				# 텍스트만 출력

In [ ]:
tag_list = soup.select("body p > a")		# <body><p> 태그 하위의 <a> 태그 추출
for tag in tag_list:						# tag_list 내 데이터를 하나씩 읽어오기
    print(tag.text)							# 텍스트만 출력

In [ ]:
tag_list = soup.select("body p a")			# <body><p> 태그 하위의 <a> 태그 추출하고, 그 하위의 모든 태그도 추출
for tag in tag_list:						# tag_list 내 데이터를 하나씩 읽어오기
    print(tag.text)							# 텍스트만 출력

In [ ]:
tag_list = soup.select(".footage_cls")		# footage_cls 클래스 추출
for tag in tag_list:						# tag_list 내 데이터를 하나씩 읽어오기
    print(tag.text)							# 텍스트만 출력

In [ ]:
tag_list = soup.select("#company")			# id가 company인 태그 추출
for tag in tag_list:						# tag_list 내 데이터를 하나씩 읽어오기
    print (tag.text)						# 텍스트만 출력

In [ ]:
tag_list = soup.select("a[href]")			# a 태그 중 속성 값 href 를 갖는 태그 추출
for tag in tag_list:						# tag_list 내 데이터를 하나씩 읽어오기
    print(tag.text, tag.attrs["href"])		# 텍스트와 URL 출력

## 서점 베스트셀러 정보 가져오기
교보문고 베스트셀러 웹 페이지인 http://www.kyobobook.co.kr/bestSellerNew/bestseller.laf 에 접속

In [ ]:
# 패키지 불러오기
import requests									# 웹 리소스를 가져오는 모듈
from bs4 import BeautifulSoup as bs				#데이터 변환(파싱) 및 원하는 태그 추출 시 사용
import pandas as pd								#데이터 프레임

# 교보문고의 월간 베스트셀러 웹 페이지 가져오기
res = requests.get("http://www.kyobobook.co.kr/bestSellerNew/bestseller.laf?range = \
        1&kind=2&orderClick = DAB&mallGb=KOR&linkClass=A")
book = bs(html, "html.parser")					# HTML 파싱
#print(book)

# 도서 상세 페이지 주소 추출하기
book_list = []									# book_list 리스트 변수 만들기
for book_detail in book.select("div.detail"):	# book에서 class=detail 태그 가져오기
    book_urls = book_detail.select_one("div.title > a").attrs["href"]   # 상세 주소를 book_urls에 저장
    book_list.append(book_urls)					# book_list 변수에 주소를 하나씩 추가
    #print(book_list)

# 도서별 상세 데이터 추출하기
best = pd.DataFrame(columns=["도서","저자","가격","url"])		# 저장할 데이터프레임 만들기
for index, book_url in enumerate(book_list):				# 도서 상세 페이지를 하나씩 반환
    res = requests.get(book_url)							# 도서 상세 페이지 리소스 가져오기
    html = res.content
    best_book = bs(html, "html.parser", from_encoding = "cp949")		# HTML 파싱
    title = best_book.select_one("h1.title strong").text.strip()		# 도서명 추출
    author = best_book.select_one("span.name a").text.strip()			# 저자명 추출
    price = best_book.select_one("span.sell_price strong").text.strip()	# 가격 추출    
    best.loc[index+1] = (title, author, price, book_url)				# 데이터 프레임에 저장
#best.head()

# 엑셀 파일로 저장하기
best.to_excel("chapter08/bestseller.xlsx")

## 8.3 웹 브라우저 제어

### 본인의 chrome 정보를 확인한 후
- 크롬 드라이버 페이지(https://chromedriver.chromium.org/downloads) 에 접속해서 크롬 브라우저의 버전과 동일한 웹 드라이버를 다운로드


### selenium

In [ ]:
!pip install selenium

### 웹 로드 및 HTML 소스 가져오기

In [ ]:
import selenium						# selenium 모듈 불러오기
from selenium import webdriver		# webdriver 모듈 불러오기

In [ ]:
URL = "https://www.naver.com"				# 네이버 URL을 URL 변수에 저장
driver = webdriver.Chrome("chromedriver")	# driver 객체 생성 
driver.get(URL)								# URL에 저장된 주소를 크롬에서 열기

In [ ]:
print(driver.current_url)				# 현재 URL 출력

In [ ]:
driver.close()

### HTML 소스 가져오기

In [ ]:
# chrom 으로 Naver 자동 접속하기
import selenium								# selenium 모듈 불러오기
from bs4 import BeautifulSoup				# BeautifulSoup 모듈 불러오기
from selenium import webdriver				# webdriver 모듈 불러오기
URL = "https://www.naver.com"				# 네이버 URL을 URL 변수에 저장
driver = webdriver.Chrome("chromedriver")	# driver 객체 생성 
driver.get(URL)								# URL에 저장된 주소를 크롬에서 열기

# HTML 소스 파싱하기
html = driver.page_source  					# HTML 소스 가져오기
soup = BeautifulSoup(html, "html.parser")	# 데이터 변환(파싱)

# 텍스트 출력하기
tag_list = soup.select("body p")			# <body> 내 <p> 태그 추출
for tag in tag_list:						# tag_list 내 데이터를 하나씩 읽어오기
    print(tag.text)							# 텍스트만 출력

In [ ]:
driver.close()    

## 웹 브라우저 제어하기

### [Naver 로그인] 버튼 자동으로 클릭하기

In [ ]:
import selenium													# selenium 모듈 불러오기
from selenium import webdriver									# webdriver 모듈 불러오기
from selenium.webdriver.common.by import By						# selenium의 by 클래스 불러오기
URL = "https://www.naver.com"									# 네이버 URL을 URL 변수에 저장
driver = webdriver.Chrome("chromedriver")						# driver 객체 생성 
driver.get(URL)													# URL에 저장된 주소를 크롬에서 열기
btn = driver.find_element(By.XPATH, '//*[@id="account"]/a')		# 네이버 로그인 버튼 선택
btn.click()														# 로그인 버튼 클릭

### 네이버에서 방탄소년단 검색하기

In [ ]:
import selenium													# selenium 모듈 불러오기
from selenium import webdriver									# webdriver 모듈 불러오기
from selenium.webdriver.common.by import By						# selenium의 by 클래스 불러오기
URL = "https://www.naver.com"									# 네이버 URL을 URL 변수에 저장
driver = webdriver.Chrome("chromedriver")						# driver 객체 생성 
driver.get(URL)													# URL에 저장된 주소를 크롬에서 열기
input_tag = driver.find_element(By.CSS_SELECTOR, "#query")		# 검색창으로 이동하여
input_tag.send_keys("방탄소년단")								# 검색창에 문자 입력
input_tag.send_keys("\n")										# Enter 키 실행
driver.implicitly_wait(3)										# 최대 3초간 대기(3초가 지나면 다음 코드 수행)

In [ ]:
driver.close() 																# 웹 드라이버 종료